In [1]:
!pip install -U scipy==1.8
!git clone https://github.com/Marcos-L/MI-EEG-ClassMeth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 34.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 64.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.17 requires scipy>=1.10.0, but you have scipy 1.8.0 which is incompatible.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0

In [2]:
cd /kaggle/working/MI-EEG-ClassMeth

/kaggle/working/MI-EEG-ClassMeth


In [3]:
from MI_EEG_ClassMeth.MIfunctions import Window_band_CSP_eppoch, flatt, elastic_net_feats

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
cd /kaggle/working/

/kaggle/working


In [ ]:
from shutil import copy2
import pickle

import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import cohen_kappa_score, roc_auc_score, accuracy_score, \
                            f1_score, recall_score, precision_score, make_scorer
from scipy.signal import freqz, filtfilt, resample
from scipy.signal import butter as bw
from joblib import dump, load

In [6]:
def butterworth_digital_filter(X, N, Wn, btype, fs, axis=-1, padtype=None, padlen=0, method='pad', irlen=None):
  """
  Apply digital butterworth filter
  INPUT
  ------
  1. X: (D array)
    array with signals.
  2. N: (int+)
    The order of the filter.
  3. Wn: (float+ or 1D array)
    The critical frequency or frequencies. For lowpass and highpass filters, Wn is a scalar; for bandpass and bandstop filters, Wn is a length-2 vector.
    For a Butterworth filter, this is the point at which the gain drops to 1/sqrt(2) that of the passband (the “-3 dB point”).
    If fs is not specified, Wn units are normalized from 0 to 1, where 1 is the Nyquist frequency (Wn is thus in half cycles / sample and defined as 2*critical frequencies / fs). If fs is specified, Wn is in the same units as fs.
  4. btype: (str) {‘lowpass’, ‘highpass’, ‘bandpass’, ‘bandstop’}
    The type of filter
  5. fs: (float+)
    The sampling frequency of the digital system.
  6. axis: (int), Default=1.
    The axis of x to which the filter is applied.
  7. padtype: (str) or None, {'odd', 'even', 'constant'}
    This determines the type of extension to use for the padded signal to which the filter is applied. If padtype is None, no padding is used. The default is ‘odd’.
  8. padlen: (int+) or None, Default=0
    The number of elements by which to extend x at both ends of axis before applying the filter. This value must be less than x.shape[axis] - 1. padlen=0 implies no padding.
  9. method: (str), {'pad', 'gust'}
    Determines the method for handling the edges of the signal, either “pad” or “gust”. When method is “pad”, the signal is padded; the type of padding is determined by padtype
    and padlen, and irlen is ignored. When method is “gust”, Gustafsson’s method is used, and padtype and padlen are ignored.
  10. irlen: (int) or None, Default=nONE
    When method is “gust”, irlen specifies the length of the impulse response of the filter. If irlen is None, no part of the impulse response is ignored.
    For a long signal, specifying irlen can significantly improve the performance of the filter.
  OUTPUT
  ------
  X_fil: (D array)
    array with filtered signals.
  """
  b, a = bw(N, Wn, btype, analog=False, output='ba', fs=fs)
  return filtfilt(b, a, X, axis=axis, padtype=padtype, padlen=padlen, method=method, irlen=irlen)

class TimeFrequencyRpr(BaseEstimator, TransformerMixin):
  """
  Time frequency representation of EEG signals.

  Parameters
  ----------
    1. sfreq:  (float) Sampling frequency in Hz.
    2. f_bank: (2D array) Filter banks Frequencies. Default=None
    3. vwt:    (2D array) Interest time windows. Default=None
  Methods
  -------
    1. fit(X, y=None)
    2. transform(X, y=None)
  """
  def __init__(self, sfreq, f_bank=None, vwt=None):
    self.sfreq = sfreq
    self.f_bank = f_bank
    self.vwt = vwt
# ------------------------------------------------------------------------------

  def _validation_param(self):
    """
    Validate Time-Frequency characterization parameters.
    INPUT
    -----
      1. self
    ------
      2. None
    """
    if self.sfreq <= 0:
      raise ValueError('Non negative sampling frequency is accepted')


    if self.f_bank is None:
      self.flag_f_bank = False
    elif self.f_bank.ndim != 2:
      raise ValueError('Band frequencies have to be a 2D array')
    else:
      self.flag_f_bank = True

    if self.vwt is None:
      self.flag_vwt = False
    elif self.vwt.ndim != 2:
      raise ValueError('Time windows have to be a 2D array')
    else:
      self.flag_vwt = True

# ------------------------------------------------------------------------------
  def _filter_bank(self, X):
    """
    Filter bank Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_f: (4D array) set of filtered EEG signals, shape (trials, channels, time_samples, frequency_bands)
    """
    X_f = np.zeros((X.shape[0], X.shape[1], X.shape[2], self.f_bank.shape[0])) #epochs, Ch, Time, bands
    for f in np.arange(self.f_bank.shape[0]):
      X_f[:,:,:,f] = butterworth_digital_filter(X, N=5, Wn=self.f_bank[f], btype='bandpass', fs=self.sfreq)
    return X_f

# ------------------------------------------------------------------------------
  def _sliding_windows(self, X):
    """
    Sliding Windows Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_w: (4D array) shape (trials, channels, window_time_samples, number_of_windows)
    """
    window_lenght = int(self.sfreq*self.vwt[0,1] - self.sfreq*self.vwt[0,0])
    X_w = np.zeros((X.shape[0], X.shape[1], window_lenght, self.vwt.shape[0]))
    for w in np.arange(self.vwt.shape[0]):
        X_w[:,:,:,w] = X[:,:,int(self.sfreq*self.vwt[w,0]):int(self.sfreq*self.vwt[w,1])]
    return X_w

# ------------------------------------------------------------------------------
  def fit(self, X, y=None):
    """
    fit.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
      2. y: (1D array) target labels. Default=None
    OUTPUT
    ------
      1. None
    """
    pass

# ------------------------------------------------------------------------------
  def transform(self, X, y=None):
    """
    Time frequency representation of EEG signals.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, times)
    OUTPUT
    ------
      1. X_wf: (5D array) Time-frequency representation of EEG signals, shape (trials, channels, window_time_samples, number_of_windows, frequency_bands)
    """
    self._validation_param()     #Validate sfreq, f_freq, vwt

    #Avoid edge effects of digital filter, 1st:fbk, 2th:vwt
    if self.flag_f_bank:
        X_f = self._filter_bank(X)
    else:
        X_f = X[:,:,:,np.newaxis]

    if self.flag_vwt:
      X_wf = []
      for f in range(X_f.shape[3]):
        X_wf.append(self._sliding_windows(X_f[:,:,:,f]))
      X_wf = np.stack(X_wf, axis=-1)
    else:
      X_wf = X_f[:,:,:,np.newaxis,:]

    return X_wf

def kappa(y_true, y_pred):
    if len(y_true.shape) == 1:
        return cohen_kappa_score(y_true, np.argmax(y_pred, axis = 1))
    else:
        return cohen_kappa_score(np.argmax(y_true, axis = 1), np.argmax(y_pred, axis = 1))

In [ ]:
import numpy as np
import mne
import pandas as pd


# ----------------------------------------------------------------------
def apply_along_epochs(func1d, epochs):
    """"""
    epochs_ = epochs.copy()

    data = np.apply_along_axis(func1d, -1, epochs_._data)

    epochs_._data = data

    return epochs_


# ----------------------------------------------------------------------
def get_best_montage(channels):
    """"""
    data = {}
    target = set([_.lower() for _ in channels])

    for montage_name in mne.channels.get_builtin_montages():
        montage = mne.channels.make_standard_montage(montage_name)
        count = len(target.intersection(
            set([_.lower() for _ in montage.ch_names])))
        missings = target.difference(set([_.lower() for _ in montage.ch_names]))
        missings_name = [channels[[_.lower() for _ in channels].index(nl)]
                         for nl in missings]

        data.setdefault('count', []).append(count)
        data.setdefault('missings', []).append(len(missings))
        data.setdefault('missings channels', []).append(missings_name)
        data.setdefault('montage', []).append(montage_name)

    df = pd.DataFrame.from_dict(data)
    return df.sort_values('count', ascending=False, ignore_index=True)

In [8]:
def load_PAIN(db,sbj,f_bank,vwt,new_fs):

    channels_names = np.array(['Fp1','Fp2',
                      'F3','F4','C3','C4','P3','P4','O1','O2','F7','F8',
                      'T7','T8','P7','P8','Fz','Cz','Pz','Oz',
                      'FC1','FC2','CP1','CP2','FC5','FC6','CP5','CP6',
                      'TP9','TP10','LE','RE','P1','P2','C1','C2',
                      'FT9','FT10','AF3','AF4','FC3','FC4','CP3','CP4','PO3','PO4',
                      'F5','F6','C5','C6','P5','P6','PO9','Iz','FT7','FT8',
                      'TP7','TP8','PO7','PO8','Fpz','PO10','CPz','POz',
                      'Ne','Ma','Ext','ECG'])
    
    with open('{}BMOP_Motor_S{}.pkl'.format(db,'0' + str(sbj) if sbj < 10 else sbj), 'rb') as f:
        data = pickle.load(f)
        
    X = data['X']  # trials, channels, time
    y = data['y']
    sex = data['sex'].ravel()
    age = data['age'].ravel()
    fs = float(data['fs'])
    
    tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)
    
    #Read electrode positions to load the best standard montage-MNE
    best_montages = get_best_montage(channels_names)
    montage = best_montages.iloc[0]['montage']
    no_channels = np.array(best_montages.iloc[0]['missings channels'])
    channels_to_remove = np.array([np.argwhere(channels_names==no)[0] for no in no_channels])[:,0]

    #Delete the missing channels the original array respecting the positions
    channels_names = np.delete(channels_names, channels_to_remove)
    X = np.delete(X, channels_to_remove, axis=1)

    #Number channels does not match with the dimension of X, 
    #thus the last channel is discarded because it has weird amplitudes
    X = X[:,:-1,:]

    info = mne.create_info(list(channels_names), sfreq=fs, ch_types="eeg")
    info.set_montage(montage)
    info

    event_id = {
        'pain/high':2,
        'resting':3,
        }

    events = [[i, 1, cls[0]] for i, cls in enumerate(y)]
    tmin = 0
    
    epochs = mne.EpochsArray(X, info, events=events, tmin=tmin, event_id=event_id)
    X = epochs.get_data()
    y = y-2
    X = np.squeeze(tf_repr.transform(X))
                             
    #Resampling
    if new_fs != fs:
        # print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)
    return X,y,age,sex,fs

In [ ]:
import os 

seed=23
folds=5
epochs_train = 500

data_set='PAIN'

model_name = f'GFC'

save_folder = os.path.join('CSP_Loso', 'CSP_Loso')

n_subjects = 51

PATH = f'{os. getcwd()}/{save_folder}'

In [10]:
f_frec =  np.array([[4,8],[8,13],[13,32],[30,60]])

tau =  2
overlap=1-0.5
ti = 0.5
tf_ = 2.5
tti = np.arange(ti,tf_-tau+(tau*overlap),tau*overlap)
ttf = np.arange(ti+tau,tf_+tau*overlap,tau*overlap)
vtw = np.array([tti,ttf]).T

In [11]:
training_details = {'Females': {'Subjects': [1., 10., 12., 23., 29., 46.]}, 'Males': {'Subjects': [15., 19., 21., 25., 27.]}}

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut

subjects = np.arange(n_subjects)+1

db = '../input/brain-mediators-of-pain-motor/'

num_class = 2

load_args = dict(db = db,
            f_bank = np.asarray([[4., 60.]]),
            vwt = np.asarray([[0.5,2.5]]),
            new_fs = 500.0)

genders = list(training_details.keys())

print("Starting experiment...\n")

for gender in genders:
    gender_subs = training_details[gender]["Subjects"]
    training_details[gender]["Groups"] = [] 
    g = 0
    
    for sbj in gender_subs:
        print(f"Loading subject: {int(sbj)}\n")
        load_args['sbj'] = int(sbj) 

        if (sbj == gender_subs[0]):
            X_train, Y_train, _, sex, _ = load_PAIN(**load_args)
            g+=1
            training_details[gender]["Groups"] += [g] * len(X_train)
        else:
            X_train_, Y_train_, _, sex, _ = load_PAIN(**load_args)
            X_train = np.concatenate((X_train, X_train_), axis = 0)
            Y_train = np.concatenate((Y_train, Y_train_), axis = 0)
            
            g+=1
            training_details[gender]["Groups"] += [g] * len(X_train_)
        print("\n")

    Y_train = np.array([y[0] for y in Y_train])

    # ---CSP
    steps=[('CSP_window_time',Window_band_CSP_eppoch(fs=128., vtw=vtw, f_frec=f_frec)),
            ('flat',flatt()),
            ('stand',StandardScaler()),
            ('proy',elastic_net_feats()),
            ('cla', LDA())]

    method = Pipeline(steps, memory='/kaggle/tmp/datospipeline')

    parameters ={'CSP_window_time__ncomp':[6],
                 'proy__alpha':[0, 0.001, 0.1, 1], #0, 0.001, 0.1, 1
                 'proy__l1_ratio':[0, 0.5, 1]}

    scores = {'acc': 'accuracy',
              'kappa': make_scorer(cohen_kappa_score, greater_is_better=True),
              'auc': make_scorer(roc_auc_score, greater_is_better=True)}

    logo = LeaveOneGroupOut()

    grid_search = GridSearchCV(method, parameters, cv=logo, verbose=0,
                               scoring=scores, refit='acc', n_jobs=1)

    grid_search.fit(X = X_train, y = Y_train, groups = training_details[gender]["Groups"])

    grid_search.cv_results_['best_index_'] = grid_search.best_index_

    full_path = os.path.join(os.getcwd(), f'FBCSPMotorLosoGender', gender)

    try:
        os.makedirs(full_path)
    except:
        pass

    with open(full_path + f'/FBCSP_{gender}.p','wb') as f:
        pickle.dump(grid_search.cv_results_, f)     
    
    print(f'Acc: {np.round(grid_search.best_score_*100,4)}')
    
    # ---CSP
    print('-------------------------------------------------------------\n\n')

Starting experiment...

Loading subject: 1

Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


Loading subject: 10

Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


Loading subject: 12

Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


Loading subject: 23

Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


Loading subject: 29

Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


Loading subject: 46

Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


Acc: 63.75
-------------------------------------------------------------


Loading subject: 15



Loading subject: 19



Loading subject: 21



Loading subject: 25



Loading subject: 27



Acc: 59.7632
